In [1]:
%matplotlib inline

#- import required python libraries
import netCDF4
import sys, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import glofrim.glofrim_lib as glib
from os.path import join, isfile, abspath, dirname, basename, normpath
from configparser import ConfigParser
from glofrim.glofrim_lib import configread

In [2]:
from glofrim import Glofrim, PCR, LFP
cbmi = Glofrim()

In [5]:
#- provide file with local paths to model objects
env_fn = r'../../environment.env'
config = configread(env_fn)
root_dir = config.get('models', 'root_dir')

FileNotFoundError: [Errno 2] No such file or directory: '../../environment.env'

In [3]:
#- convert model configs to dictionary
_cf = ConfigParser(inline_comment_prefixes=('#'))
env = glib.configread(abspath(env_fn), encoding='utf-8', cf=_cf)
glib.config2dict(env)

NameError: name 'env_fn' is not defined

In [4]:
#- provide file with coupling settings
# config_fn = join(root_dir, 'glofrim_PCR2CMF.ini')
# config_fn = join(root_dir, 'glofrim_PCR2DFM.ini')
# config_fn = join(root_dir, 'glofrim_WFL2DFM.ini')
root_dir = '/home/hcwinsemius/git/glofrim'
config_fn = join(root_dir, 'glofrim.ini')
config_fn

'/home/hcwinsemius/git/glofrim/glofrim.ini'

In [5]:
#- provide locations where to extract simulated discharge
x_obs_coord = 10.89 # set to coords of Neu-Darchau
y_obs_coord = 53.23

In [6]:
#- create coupled GLOFRIM model and initialize configs
try:
    cbmi.initialize_config(config_fn) #, env_fn=env_fn)
except Exception as e:
    print(e)
    sys.exit('configs could not be initialized')

2020-03-24 14:10:01,713 - GLOFRIM - INFO - Reading ini file..
2020-03-24 14:10:01,720 - GLOFRIM.LFP - INFO - Config initialized
2020-03-24 14:10:01,721 - GLOFRIM - INFO - Parsing exchanges..


In [7]:
#- provide start and end date
t_start = datetime(2000,1,1)
t_end = datetime(2000,1,3)
cbmi.set_start_time(t_start)
cbmi.set_end_time(t_end)
try:
    t_start == cbmi.get_start_time()
    t_end == cbmi.get_end_time()
except:
    sys.exit('start or end time differ with set_var and get_var')

In [9]:
t_end

datetime.datetime(2000, 1, 3, 0, 0)

In [10]:
#- create coupled GLOFRIM model and initialize models
try:
    cbmi.initialize_model()
except Exception as e:
    print(e)
    sys.exit('models could not be initialized')

2020-03-24 14:10:38,685 - GLOFRIM - INFO - set coupled grids for: SpatialCoupling object from LFP to LFP based on grid_2_grid method
2020-03-24 14:10:38,688 - GLOFRIM.LFP - INFO - Getting rgrid info based on dem.asc
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "<ipython-input-10-1073c4172f3c>", line 3, in <module>
    cbmi.initialize_model()
  File "/home/hcwinsemius/git/glofrim/glofrim/glofrim_bmi.py", line 353, in initialize_model
    self._set_spatial_coupling()
  File "/home/hcwinsemius/git/glofrim/glofrim/glofrim_bmi.py", line 333, in _set_spatial_coupling
    item[1]['coupling'].couple()
  File "/home/hcwinsemius/git/glofrim/glofrim/spatial_coupling.py", line 74, in couple
    self.from_bmi.get_grid() # make sure grid property is set
  File "/home/hcwinsemius/git/glofrim/glofrim/lfp_bmi.py", line 201, in get_grid
    self.grid.set_1d(nodes=np.array(zip(x, y)), links=None, inds=inds)
  File "/home/hcwinsemius/git/glofrim/glofrim/grids.py", line 55, in set_1d
    self._1d = Network1D(nodes, links=links, inds=inds, fill_value=fill_value, ds_idx=ds_idx, us_idx=us_idx)
  File "/home/hcwinsemius/git/glofrim/glofrim/grids.py", line 322, in __init__
    assert self.nnodes == self.inds.size
AssertionEr

SystemExit: models could not be initialized

/home/hcwinsemius/miniconda3/envs/barotse/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#- get corresponding indices for models
obs_ind_PCR = cbmi.index(x_obs_coord, y_obs_coord, 'PCR', in1d=False)
print obs_ind_PCR
obs_ind_LFP = cbmi.index(x_obs_coord, y_obs_coord, 'LFP', in1d=True)
print obs_ind_LFP

In [ ]:
#- update models until end time is reached
#- also, read out discharge data at specified observation point
try:
    i = 0
    while i < 10:
#     while t_start < t_end:
        cbmi.update()
        q_sim_PCR = cbmi.get_value_at_indices('PCR.discharge', obs_ind_PCR)
        q_sim_LFP = cbmi.get_value_at_indices('LFP.Qx', obs_ind_DFM) 
        print q_sim_PCR, q_sim_LFP
        i += 1
except Exception as e:
    print e
    sys.exit('something is going wrong in updating - please check!')

In [ ]:
#- finalize all models
try:
    cbmi.finalize()
except Exception as e:
    print e
    sys.exit('models could not be finalized')